<a href="https://colab.research.google.com/github/waishun78/hungry-rag/blob/main/notebooks/python_elasticsearch_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

In [ ]:
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm
  import datetime
  from elasticsearch import helpers

except Exception as e:
  print(f"error: {e}")

In [ ]:
# Download & extract Elasticsearch 7.0.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
# Creating daemon instance of elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(20)

In [ ]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon      9218    8930  4 13:32 ?        00:00:42 /content/elasticsearch-7.0.0/jdk/bin/java -Xms1g
daemon      9323    9218  0 13:32 ?        00:00:00 /content/elasticsearch-7.0.0/modules/x-pack-ml/p
root       13278   13276  0 13:48 ?        00:00:00 grep elasticsearch


In [ ]:
# Check if elasticsearch is running
!curl -sX GET "localhost:9200/"

{
  "name" : "11b2b573d2cc",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "fuNY1aCxT0upmc7BIp76Kw",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
!curl -sX GET "localhost:9200/food_index"

{"food_index":{"aliases":{},"mappings":{"properties":{"article":{"type":"text"},"doc":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"highlights":{"type":"text"},"snippet":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"title":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"url":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}}}},"settings":{"index":{"creation_date":"1699277012276","number_of_shards":"1","number_of_replicas":"0","uuid":"M6-IcPsKS-S3hEBvgh4hDw","version":{"created":"7000099"},"provided_name":"food_index"}}}}

In [ ]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch

In [ ]:
es.ping()

True

In [ ]:
# Importing test dataset
dataset = pd.read_csv("documents_to_index.csv")
print(f"shape of dataset: {dataset.shape}")
dataset.head()

shape of dataset: (699, 4)


,url,title,snippet,doc
0,https://www.timeout.com/singapore/restaurants/...,50 Best Restaurants in Singapore You Must Try,31 Aug 2023 —,"Awesome, you're subscribed!Thanks for subscri..."
1,https://thehoneycombers.com/singapore/best-res...,57 best restaurants in Singapore: Your ultimat...,24 Oct 2023 —,This website is using a security service to pr...
2,https://www.tripadvisor.com.sg/Restaurants-g29...,THE 10 BEST Restaurants in Singapore,The best restaurants in Singapore include: Ent...,NaN
3,https://www.theworlds50best.com/discovery/site...,The best restaurants and bars in Singapore,The best restaurants and bars in Singapore. Br...,Explore the best restaurants and bars around t...
4,https://www.chope.co/singapore-restaurants/pag...,100 Best Restaurants in Singapore,With all sorts of the best restaurants in Sing...,Welcome to the only food bible you’ll need for...


In [ ]:
# Define settings & mappings of Elasticsearch index
Settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "url":{
                "type":"text"
            },
            "title":{
                "type":"text"
            },
            "snippet":{
                "type":"text"
            },
            "doc":{
                "type":"text"
            }
        }
    }
}

In [ ]:
def json_formatter(dataset, index_name, index_type='_doc'):
    """
    This function is used to create JSON formatted dictionaries for Elasticsearch.

    Args:
      dataset: The dataset you want to apply this function.
      index_name: Name of the index in Elasticsearch
      index_type: Type of the index in Elasticsearch.
      Note: It is suggested to keep index_type as '_doc' since it is deprecated from version 6.
      Note: This function formats all columns of your dataset, if you want to apply this to special columns only,
      you can delete the second for loop and add your custom fields.
    """
    try:
        List = []
        columns = dataset.columns
        for idx, row in dataset.iterrows():
            dic = {}
            dic['_index'] = index_name
            # dic['_type'] = index_type
            source = {}
            for i in dataset.columns:
                source[i] = row[i]
            dic['_source'] = source
            List.append(dic)
        return List

    except Exception as e:
        print("There is a problem: {}".format(e))

In [ ]:
es.indices.exists(index='food_index')

True

In [ ]:
# es.indices.delete(index='food_index', ignore=[400, 404])

In [ ]:
MY_INDEX = es.indices.create(index="food_index", ignore=[400,404], body=Settings)
MY_INDEX

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 467, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 462, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/elasticsearch/connection/http_urllib

ConnectionTimeout: ignored

In [ ]:
json_Formatted_dataset = json_formatter(dataset=dataset, index_name='food_index')
json_Formatted_dataset[0]

{'_index': 'food_index',
 '_type': '_doc',
 '_source': {'url': 'https://www.timeout.com/singapore/restaurants/the-50-best-restaurants-in-singapore',
  'title': '50 Best Restaurants in Singapore You Must Try',
  'snippet': '31 Aug 2023 — ',
  'doc': " Awesome, you're subscribed!Thanks for subscribing! Look out for your first newsletter in your inbox soon!The best of Singapore for free.Sign up for our email to enjoy Singapore without spending a thing (as well as some options when you’re feeling flush).Déjà vu! We already have this email. Try another?By entering your email address you agree to our Terms of Use and Privacy Policy and consent to receive emails from Time Out about news, events, offers and partner promotions. Awesome, you're subscribed!Thanks for subscribing! Look out for your first newsletter in your inbox soon!Love the mag?Our newsletter hand-delivers the best bits to your inbox. Sign up to unlock our digital magazines and also receive the latest news, events, offers and pa

In [ ]:
# For importing Data to elasticsearch we use elasticsearch's bulk API from elasticsearch.helpers
try:
    res = helpers.bulk(es, json_Formatted_dataset[0])
    print("successfully imported to elasticsearch.")
except Exception as e:
    print(f"error: {e}")

error: RequestError(400, 'action_request_validation_exception', 'Validation Failed: 1: index is missing;2: index is missing;3: index is missing;')


In [74]:
# Get 10 sample of data
query = es.search(
    index="food_index",
    body={
      "size":10,
      "query": {
        "match_all":{}
      }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

,_index,_type,_id,_score,_source.url,_source.title,_source.snippet,_source.doc
0,food_index,_doc,Yx3YpIsB2sEoj8J0McOq,1.0,https://www.timeout.com/singapore/restaurants/...,50 Best Restaurants in Singapore You Must Try,31 Aug 2023 —,"Awesome, you're subscribed!Thanks for subscri..."
1,food_index,_doc,ZB3YpIsB2sEoj8J0McOq,1.0,https://thehoneycombers.com/singapore/best-res...,57 best restaurants in Singapore: Your ultimat...,24 Oct 2023 —,This website is using a security service to pr...
2,food_index,_doc,Zh3YpIsB2sEoj8J0McOq,1.0,https://www.theworlds50best.com/discovery/site...,The best restaurants and bars in Singapore,The best restaurants and bars in Singapore. Br...,Explore the best restaurants and bars around t...
3,food_index,_doc,Zx3YpIsB2sEoj8J0McOq,1.0,https://www.chope.co/singapore-restaurants/pag...,100 Best Restaurants in Singapore,With all sorts of the best restaurants in Sing...,Welcome to the only food bible you’ll need for...
4,food_index,_doc,aR3YpIsB2sEoj8J0McOq,1.0,https://secretsingapore.co/singapore-best-rest...,42 Best Restaurants In Singapore For Your Buck...,29 Aug 2023 —,Singapore is buzzing with hundreds of eateries...
5,food_index,_doc,ah3YpIsB2sEoj8J0McOq,1.0,https://www.theurbanlist.com/singapore/a-list/...,The Best Singapore Restaurants Every Foodie Ne...,20 Of The Best Singapore Restaurants Every Foo...,By Gracie Stewart1st Nov 2023Singapore has lon...
6,food_index,_doc,ax3YpIsB2sEoj8J0McOq,1.0,https://guide.michelin.com/sg/en/singapore-reg...,Restaurants - Singapore - MICHELIN Guide,Singapore : 1-20 of 286 restaurants · National...,Unfortunately there is no selected restaurants...
7,food_index,_doc,bB3YpIsB2sEoj8J0McOq,1.0,https://eatbook.sg/best-restaurants-singapore/,60 Best Restaurants In Singapore [2023],24 Aug 2023 —,Filter By(Change?)Singapore is one of the worl...
8,food_index,_doc,bh3YpIsB2sEoj8J0McOq,1.0,https://eatbook.sg/best-seafood-restaurants/,15 Best Seafood Restaurants In Singapore,2 Aug 2023 —,Filter By(Change?)With chilli crab being one o...
9,food_index,_doc,bx3YpIsB2sEoj8J0McOq,1.0,https://www.thebestsingapore.com/eat-and-drink...,7 Best Seafood Restaurants in Singapore Right Now,1. Palm Beach Seafood Restaurant; 2. Chin Huat...,Whether you’re craving the comforting flavors ...


In [80]:
# Complicated query
query = es.search(
    index="food_index",
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"match":{"title":"seafood"}}
                ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

,_index,_type,_id,_score,_source.url,_source.title,_source.snippet,_source.doc
0,food_index,_doc,bh3YpIsB2sEoj8J0McOq,0.861853,https://eatbook.sg/best-seafood-restaurants/,15 Best Seafood Restaurants In Singapore,2 Aug 2023 —,Filter By(Change?)With chilli crab being one o...
1,food_index,_doc,cR3YpIsB2sEoj8J0McOq,0.861853,https://thehoneycombers.com/singapore/seafood-...,16 stunning seafood restaurants in Singapore,23 Jun 2022 —,This website is using a security service to pr...
2,food_index,_doc,cx3YpIsB2sEoj8J0McOq,0.813328,https://finestservices.com.sg/blog/seafood-res...,10 Best Seafood Restaurant In Singapore [2023],10 Best Seafood Restaurants in Singapore · Orc...,Picking at a piece of crab may be therapeutic:...
3,food_index,_doc,dB3YpIsB2sEoj8J0McOq,0.813328,https://www.funempire.com/sg/blog/seafood-rest...,19 Best Seafood Restaurant Singapore Options [...,The best seafood restaurants in Singapore incl...,When it comes to savouring the flavours of the...
4,food_index,_doc,dh3YpIsB2sEoj8J0McOq,0.813328,https://www.ladyironchef.com/tag/best-seafood-...,Tag Archives: Best Seafood Restaurants In Sing...,26 Jun 2022 —,Orchid Live Seafood at HomeTeamNS Khatib is an...
5,food_index,_doc,bx3YpIsB2sEoj8J0McOq,0.769976,https://www.thebestsingapore.com/eat-and-drink...,7 Best Seafood Restaurants in Singapore Right Now,1. Palm Beach Seafood Restaurant; 2. Chin Huat...,Whether you’re craving the comforting flavors ...
6,food_index,_doc,cB3YpIsB2sEoj8J0McOq,0.769976,https://secretsingapore.co/seafood-restaurants...,Seafood Restaurants Singapore: 10 Of The Best ...,3 Aug 2023 —,Sink your teeth into Singapore's seductive sea...
7,food_index,_doc,ch3YpIsB2sEoj8J0McOq,0.769976,https://citynomads.com/best-seafood-restaurant...,Best Seafood Restaurants in Singapore: Where t...,14 Aug 2023 —,"City NomadsPUBLISHED August 14th, 2023 06:00 a..."
8,food_index,_doc,dR3YpIsB2sEoj8J0McOq,0.695801,https://www.lifestyleasia.com/sg/dining/best-s...,8 best seafood restaurants in Singapore for th...,3 Nov 2022 —,"For most Singaporeans, seafood is something we..."


In [82]:
# More complicated query
query = es.search(
    index="food_index",
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"multi_match":{
                            "query":"korean",
                            "fields":["title","doc"]
                        }}
                ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

,_index,_type,_id,_score,_source.url,_source.title,_source.snippet,_source.doc
0,food_index,_doc,Zx3YpIsB2sEoj8J0McOq,2.587421,https://www.chope.co/singapore-restaurants/pag...,100 Best Restaurants in Singapore,With all sorts of the best restaurants in Sing...,Welcome to the only food bible you’ll need for...
